In [12]:
from sklearn.linear_model import Lasso

import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error, make_scorer, r2_score

from sklearn.svm import SVR

# Set random seed 
RSEED = 42

In [2]:
final_data = pd.read_csv("data/wrangled_data.csv")

In [3]:
del final_data['date_caught']

In [4]:
# Select X and y features
X = final_data.drop(['capture_number'], axis = 1)
y = final_data['capture_number']


# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True, stratify=y)

# Check the shape of the data sets
print("X_train:", X_train.shape)  
print("y_train:", y_train.shape)  
print("X_test:", X_test.shape) 
print("y_test:", y_test.shape)  

X_train: (12643, 40)
y_train: (12643,)
X_test: (5419, 40)
y_test: (5419,)


In [5]:
def evaluate_rmse(y_true, y_pred, ndigits=3):
    """ Prints the RMSE (root mean squared error) of y_pred in relation to y_true"""
    rmse = mean_squared_error(y_true, y_pred, squared=False )
    print("Number of predictions: ", len(y_pred))
    print("RMSE: ", round(rmse, ndigits))
    return rmse

In [13]:
# Standardize the features (optional but recommended)
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

In [27]:
# Create a Lasso Regression model
lasso = Lasso(alpha=0.7)  #  alpha is the regularization strength  parameter

In [29]:
# Fit the model to the training data
lasso.fit(X_train, y_train)

Lasso(alpha=0.7)

In [30]:
# Make predictions on the test data
y_pred = lasso.predict(X_test)

In [31]:
# evaluate
error = evaluate_rmse(y_test, y_pred)

Number of predictions:  5419
RMSE:  3.766


Parameter optimization:

In [41]:

# Create a parameter grid with distributions
param_dist = {
    'alpha': np.logspace(-3, 1, 100),        # Regularization strength (log scale)
    'fit_intercept': [True, False],          # Whether to calculate the intercept
    'max_iter': np.arange(100, 1001, 100),   # Maximum number of iterations
    'random_state': [42]                    # Random seed for reproducibility
}

In [32]:
# Create a Lasso regression model
lasso_model = Lasso()

In [42]:
# Create the RandomizedSearchCV object
random_search = RandomizedSearchCV(
    lasso_model, param_distributions=param_dist, n_iter=100, cv=5, scoring='neg_mean_squared_error', random_state=42)


In [43]:
# Fit the random search to your data
random_search.fit(X_train, y_train)

c:\Users\49176\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.676e+04, tolerance: 3.522e+01
  model = cd_fast.enet_coordinate_descent(
c:\Users\49176\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.698e+04, tolerance: 3.503e+01
  model = cd_fast.enet_coordinate_descent(
c:\Users\49176\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sca

RandomizedSearchCV(cv=5, estimator=Lasso(), n_iter=100,
                   param_distributions={'alpha': array([1.00000000e-03, 1.09749877e-03, 1.20450354e-03, 1.32194115e-03,
       1.45082878e-03, 1.59228279e-03, 1.74752840e-03, 1.91791026e-03,
       2.10490414e-03, 2.31012970e-03, 2.53536449e-03, 2.78255940e-03,
       3.05385551e-03, 3.35160265e-03, 3.67837977e-03, 4.03701726e-03,
       4.43062146e-03, 4.8626...
       2.47707636e+00, 2.71858824e+00, 2.98364724e+00, 3.27454916e+00,
       3.59381366e+00, 3.94420606e+00, 4.32876128e+00, 4.75081016e+00,
       5.21400829e+00, 5.72236766e+00, 6.28029144e+00, 6.89261210e+00,
       7.56463328e+00, 8.30217568e+00, 9.11162756e+00, 1.00000000e+01]),
                                        'fit_intercept': [True, False],
                                        'max_iter': array([ 100,  200,  300,  400,  500,  600,  700,  800,  900, 1000]),
                                        'random_state': [42]},
                   random_state=42, scoring='neg_mean_squared_error')

In [46]:
# Get the best parameters and estimator
best_params = random_search.best_params_
best_estimator = random_search.best_estimator_
best_params

Lasso(alpha=0.0010974987654930556, random_state=42)

In [47]:
y_pred = best_estimator.predict(X_test)

In [48]:
# evaluate
error = evaluate_rmse(y_test, y_pred)

Number of predictions:  5419
RMSE:  3.656


Even after hyperparameter-optimization, it is only as good as unoptimized linear regression (baseline model), 
will be dropped.